In [7]:
import pandas as pd
import numpy as np

In [72]:
def clean_applications(filename: str):
    # read the file
    df = pd.read_csv(filename) 
    # discard irrevant columns
    df = df[["State", "Year", "Domain", "Value"]]
    # convert value to numeric
    df["Value"] = pd.to_numeric(df["Value"], errors='coerce')
    # group chemical domains together
    df = df.groupby(["State", "Year", "Domain"]).agg("sum").reset_index()
    # pivot on domain
    df = df.pivot(index=["State", "Year"], columns=["Domain"], values=["Value"]).reset_index()
    # set indexes
    df.set_index(["State", "Year"], inplace=True)
    # clean column names
    df.columns = ["fungicide (lb/acre)", "herbicide (lb/acre)", "insecticide (lb/acre)", "other chemicals (lb/acre)", "fertilizer (lb/acre)"]
    # collapse indexes
    df = df.reset_index()
    df.rename(columns={"State": "state", "Year": "year"}, inplace=True)

    # set state names to lowercase
    df["state"] = df["state"].apply(lambda x: x.casefold())
    
    return df

In [73]:
# clean applications for each state and concatenate into a master dataframe
states = ["illinois", "indiana", "iowa", "minnesota", "missouri", "nebraska"]
dataframes = [clean_applications(f"../data/{state}/raw/applications.csv") for state in states]
master_applications = pd.concat(dataframes)
master_applications.to_csv("../data/all/processed/applications.csv", index=False)

In [74]:
master_applications

,state,year,fungicide (lb/acre),herbicide (lb/acre),insecticide (lb/acre),other chemicals (lb/acre),fertilizer (lb/acre)
0,illinois,1990,NaN,16.270,5.110,NaN,352.0
1,illinois,1991,NaN,17.590,3.870,NaN,342.0
2,illinois,1992,NaN,21.080,5.970,NaN,337.0
3,illinois,1993,NaN,20.460,4.800,NaN,332.0
4,illinois,1994,NaN,24.020,4.600,NaN,331.0
...,...,...,...,...,...,...,...
15,nebraska,2010,0.124,8.602,0.000,0.0,221.0
16,nebraska,2014,0.278,9.681,0.076,0.0,250.0
17,nebraska,2016,0.000,10.481,0.077,0.0,218.0
18,nebraska,2018,0.351,11.235,1.406,0.0,270.0


In [5]:
def clean_moisture(filename: str, state: str):
    # read the file
    df = pd.read_csv(filename)
    # discard irrelevant columns
    df = df[["Year", "Week Ending", "Data Item", "Value"]]
    # pivot on Data Item
    df = df.pivot(index=["Year", "Week Ending"], columns=["Data Item"], values=["Value"]).reset_index()
    # set indexes
    df.set_index(["Year", "Week Ending"], inplace=True)
    # rename columns
    df.columns = ["subsoil mositure (pct adequate)", "subsoil moisture (pct short)", "subsoil moisture (pct surplus)", "subsoil moisture (pct very short)", "topsoil moisture (pct adequate)", "topsoil moisture (pct short)", "topsoil moisture (pct surplus)", "topsoil moisture (pct very short)"]
    # collapse indexes
    df = df.reset_index().rename(columns={"Year": "year", "Week Ending": "week ending"})
    # add state column
    df["state"] = state
    # add month column
    df["week ending"] = pd.to_datetime(df["week ending"])
    df["month"] = df["week ending"].apply(lambda x: x.month_name().casefold())
    df["month"] = pd.CategoricalIndex(df["month"], categories=["april", "may", "june", "july", "august", "september", "october", "november"])
    # group by month
    df = df.groupby(["state", "year", "month"]).agg("mean").reset_index()
    # add scoring columns
    df["topsoil score"] = 1 * df["topsoil moisture (pct very short)"] + 2 * df["topsoil moisture (pct short)"] + 3 * df["topsoil moisture (pct adequate)"] + 4 * df["topsoil moisture (pct surplus)"]
    df["subsoil score"] = 1 * df["subsoil moisture (pct very short)"] + 2 * df["subsoil moisture (pct short)"]+ 3 * df["subsoil mositure (pct adequate)"] + 4 * df["subsoil moisture (pct surplus)"]
    # reorder columns
    df = df[["state", "year", "month", "subsoil score", "topsoil score"]]

    return df

In [ ]:
# clean moisture for each state and concatenate into a master dataframe
states = ["illinois", "indiana", "iowa", "minnesota", "missouri", "nebraska"]
dataframes = [clean_moisture(f"../data/{state}/raw/moisture.csv", state=state) for state in states]
master_moisture = pd.concat(dataframes)
master_moisture = master_moisture.pivot(index=["state", "year"], columns=["month"], values=["subsoil score", "topsoil score"]).reset_index()
master_moisture.set_index(["state", "year"], inplace=True)
master_moisture.columns = [' '.join(col).strip() for col in master_moisture.columns.values]
master_moisture.columns = [x.replace(" ", "-") for x in master_moisture.columns]

# create lagged columns
for col in master_moisture.columns:
    master_moisture[col + "-lagged"] = np.roll(master_moisture[col], 1)
    master_moisture.loc[0, col + "-lagged"] = None

master_moisture.reset_index(inplace=True)
master_moisture.to_csv("../data/all/processed/moisture.csv", index=False)
master_moisture

/var/folders/41/_gkgvhb94wd4156zplzr4cg00000gn/T/ipykernel_73628/851729682.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = df.groupby(["state", "year", "month"]).agg("mean").reset_index()
/var/folders/41/_gkgvhb94wd4156zplzr4cg00000gn/T/ipykernel_73628/851729682.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = df.groupby(["state", "year", "month"]).agg("mean").reset_index()
/var/folders/41/_gkgvhb94wd4156zplzr4cg00000gn/T/ipykernel_73628/851729682.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False 

,state,year,subsoil-score-april,subsoil-score-may,subsoil-score-june,subsoil-score-july,subsoil-score-august,subsoil-score-september,subsoil-score-october,subsoil-score-november,...,subsoil-score-october-lagged,subsoil-score-november-lagged,topsoil-score-april-lagged,topsoil-score-may-lagged,topsoil-score-june-lagged,topsoil-score-july-lagged,topsoil-score-august-lagged,topsoil-score-september-lagged,topsoil-score-october-lagged,topsoil-score-november-lagged
0,illinois,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,illinois,1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,314.250000,381.00,336.00,278.80,283.75,210.75,203.40,267.666667
2,illinois,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,271.000000,353.25,352.60,244.00,229.00,229.20,270.75,299.333333
3,illinois,1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,313.250000,289.25,316.80,238.25,229.20,273.00,246.75,282.800000
4,illinois,1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,350.500000,351.60,345.50,325.00,292.60,243.75,285.25,304.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,nebraska,2021,260.333333,264.00,258.00,247.5,225.8,231.5,237.6,240.50,...,207.75,205.20,295.500000,284.00,258.50,241.00,232.20,223.75,198.75,210.400000
177,nebraska,2022,177.666667,216.00,236.25,213.8,186.5,176.5,169.6,159.75,...,237.60,240.50,278.333333,275.80,264.75,256.25,235.80,252.50,254.60,255.750000
178,nebraska,2023,181.250000,190.50,198.75,215.8,228.5,205.0,210.4,207.75,...,169.60,159.75,174.000000,243.00,245.75,216.00,182.00,175.75,172.00,166.000000
179,nebraska,2024,245.000000,274.25,284.40,281.5,250.0,230.2,193.0,NaN,...,210.40,207.75,201.750000,224.00,221.00,242.00,240.25,214.75,228.80,225.000000


In [10]:
# Price-received for Missouri
df_missouri_price_received = pd.read_csv("../data/missouri/raw/price-received.csv")
df_missouri_price_received  = df_missouri_price_received[["Year", "Value"]].rename(columns = {"Year": "year", "Value": "price-received"})
df_missouri_price_received.to_csv("../data/missouri/processed/price-received.csv", index=False)


In [11]:
# Price-received for Nebraska
df_nebraska_price_received = pd.read_csv("../data/nebraska/raw/price-received.csv")
df_nebraska_price_received  = df_nebraska_price_received[["Year", "Value"]].rename(columns = {"Year": "year", "Value": "price-received"})
df_nebraska_price_received.to_csv("../data/nebraska/processed/price-received.csv", index=False)

In [25]:
# Yield for Missouri
df_missouri_yield = pd.read_csv("../data/missouri/raw/yield.csv")
df_missouri_yield  = df_missouri_yield[["Year","Data Item", "Value"]]
df_missouri_yield = df_missouri_yield.pivot(index="Year", columns="Data Item", values="Value").reset_index()
df_missouri_yield.columns = ["year", "grain yield (bu/acre)", "silage yield (bu/acre)"]
df_missouri_yield.to_csv("../data/missouri/processed/yield.csv", index=False)

In [22]:
# Yield for Nebraska
df_nebraska_yield = pd.read_csv("../data/nebraska/raw/yield.csv")
df_nebraska_yield  = df_nebraska_yield[["Year","Data Item", "Value"]]
df_nebraska_yield = df_nebraska_yield.pivot(index="Year", columns="Data Item", values="Value").reset_index()
df_nebraska_yield.columns = ["year", "grain yield (bu/acre)", "silage yield (bu/acre)"]
df_nebraska_yield.to_csv("../data/nebraska/processed/yield.csv", index=False)